# 06 - ENTRENAMIENTO DE MODELOS
## Comparación de Múltiples Algoritmos de Machine Learning

---

## Objetivo
Entrenar y comparar múltiples modelos de ML para seleccionar el mejor clasificador de enfermedades de alto costo.

**IMPORTANTE**: Este notebook maneja correctamente el encoding de variables categóricas para XGBoost.

---

## 1. IMPORTACIÓN DE LIBRERÍAS

In [ ]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# Modelos de sklearn
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# XGBoost y LightGBM
import xgboost as xgb
import lightgbm as lgb

# Métricas y validación
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

print('✓ Librerías importadas correctamente')

## 2. CARGA DE DATOS PREPROCESADOS

In [ ]:
# Cargar datos preprocesados del notebook 02
X_train = pd.read_csv('X_train_balanced.csv')
y_train = pd.read_csv('y_train_balanced.csv').values.ravel()
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv').values.ravel()

print('✓ Datos cargados exitosamente')
print(f'\n📊 DIMENSIONES:')
print(f'  Train: X={X_train.shape}, y={y_train.shape}')
print(f'  Test:  X={X_test.shape}, y={y_test.shape}')

print(f'\n📊 DISTRIBUCIÓN DE CLASES EN TRAIN:')
print(pd.Series(y_train).value_counts().sort_index())

print(f'\n📊 DISTRIBUCIÓN DE CLASES EN TEST:')
print(pd.Series(y_test).value_counts().sort_index())

## 3. ENCODING DE LA VARIABLE OBJETIVO (CRÍTICO) 🔴

**PROBLEMA**: XGBoost, LightGBM y otros algoritmos requieren que la variable objetivo sea numérica (0, 1, 2, 3), NO strings ('CANCER', 'VIH', etc.)

**SOLUCIÓN**: Usar LabelEncoder para convertir:
- CANCER → 0
- ER CRONICA → 1  
- HEMOFILIA → 2
- VIH → 3

In [ ]:
# Crear y ajustar el LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train)  # Ajustar con todas las clases del train

# Transformar AMBOS conjuntos (train y test)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print('✓ Variable objetivo encodificada correctamente')
print('\n📋 MAPEO DE CLASES:')
print('='*50)
for i, clase in enumerate(label_encoder.classes_):
    print(f'  {i} → {clase}')

print(f'\n📊 VERIFICACIÓN:')
print(f'  y_train_encoded shape: {y_train_encoded.shape}')
print(f'  y_train_encoded type: {type(y_train_encoded[0])}')
print(f'  Valores únicos: {np.unique(y_train_encoded)}')

# Guardar el encoder para uso futuro
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print('\n✓ LabelEncoder guardado: label_encoder.pkl')
print('  (Necesario para decodificar predicciones en producción)')

## 4. ENTRENAMIENTO DE MODELOS

Entrenaremos 6 modelos diferentes y compararemos su rendimiento

### 4.1 Random Forest

In [ ]:
print('\n' + '='*80)
print('MODELO 1: RANDOM FOREST')
print('='*80)

# Entrenar
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

print('⏳ Entrenando Random Forest...')
rf.fit(X_train, y_train_encoded)
print('✓ Entrenamiento completado')

# Predecir
y_pred_rf = rf.predict(X_test)

# Métricas
f1_rf = f1_score(y_test_encoded, y_pred_rf, average='macro')
f1_weighted_rf = f1_score(y_test_encoded, y_pred_rf, average='weighted')

print(f'\n📊 RESULTADOS:')
print(f'  F1-Score Macro:    {f1_rf:.4f}')
print(f'  F1-Score Weighted: {f1_weighted_rf:.4f}')

### 4.2 Gradient Boosting

In [ ]:
print('\n' + '='*80)
print('MODELO 2: GRADIENT BOOSTING')
print('='*80)

# Entrenar
gb = GradientBoostingClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    random_state=42,
    verbose=0
)

print('⏳ Entrenando Gradient Boosting...')
gb.fit(X_train, y_train_encoded)
print('✓ Entrenamiento completado')

# Predecir
y_pred_gb = gb.predict(X_test)

# Métricas
f1_gb = f1_score(y_test_encoded, y_pred_gb, average='macro')
f1_weighted_gb = f1_score(y_test_encoded, y_pred_gb, average='weighted')

print(f'\n📊 RESULTADOS:')
print(f'  F1-Score Macro:    {f1_gb:.4f}')
print(f'  F1-Score Weighted: {f1_weighted_gb:.4f}')

### 4.3 XGBoost (⭐ RECOMENDADO)

In [ ]:
print('\n' + '='*80)
print('MODELO 3: XGBOOST')
print('='*80)

# Entrenar
xgb_model = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=200,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=0
)

print('⏳ Entrenando XGBoost...')
xgb_model.fit(X_train, y_train_encoded)
print('✓ Entrenamiento completado')

# Predecir
y_pred_xgb = xgb_model.predict(X_test)

# Métricas
f1_xgb = f1_score(y_test_encoded, y_pred_xgb, average='macro')
f1_weighted_xgb = f1_score(y_test_encoded, y_pred_xgb, average='weighted')

print(f'\n📊 RESULTADOS:')
print(f'  F1-Score Macro:    {f1_xgb:.4f}')
print(f'  F1-Score Weighted: {f1_weighted_xgb:.4f}')

### 4.4 LightGBM (⭐ RECOMENDADO)

In [ ]:
print('\n' + '='*80)
print('MODELO 4: LIGHTGBM')
print('='*80)

# Entrenar
lgb_model = lgb.LGBMClassifier(
    num_leaves=31,
    learning_rate=0.05,
    n_estimators=200,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    random_state=42,
    verbose=-1
)

print('⏳ Entrenando LightGBM...')
lgb_model.fit(X_train, y_train_encoded)
print('✓ Entrenamiento completado')

# Predecir
y_pred_lgb = lgb_model.predict(X_test)

# Métricas
f1_lgb = f1_score(y_test_encoded, y_pred_lgb, average='macro')
f1_weighted_lgb = f1_score(y_test_encoded, y_pred_lgb, average='weighted')

print(f'\n📊 RESULTADOS:')
print(f'  F1-Score Macro:    {f1_lgb:.4f}')
print(f'  F1-Score Weighted: {f1_weighted_lgb:.4f}')

### 4.5 Support Vector Machine (SVM)

In [ ]:
print('\n' + '='*80)
print('MODELO 5: SUPPORT VECTOR MACHINE (SVM)')
print('='*80)

# Entrenar (puede tardar más)
svm_model = SVC(
    kernel='rbf',
    C=10,
    gamma='scale',
    probability=True,
    random_state=42,
    verbose=False
)

print('⏳ Entrenando SVM (puede tardar 5-10 minutos)...')
svm_model.fit(X_train, y_train_encoded)
print('✓ Entrenamiento completado')

# Predecir
y_pred_svm = svm_model.predict(X_test)

# Métricas
f1_svm = f1_score(y_test_encoded, y_pred_svm, average='macro')
f1_weighted_svm = f1_score(y_test_encoded, y_pred_svm, average='weighted')

print(f'\n📊 RESULTADOS:')
print(f'  F1-Score Macro:    {f1_svm:.4f}')
print(f'  F1-Score Weighted: {f1_weighted_svm:.4f}')

### 4.6 Red Neuronal (MLP)

In [ ]:
print('\n' + '='*80)
print('MODELO 6: RED NEURONAL (MLP)')
print('='*80)

# Entrenar
nn_model = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=500,
    early_stopping=True,
    random_state=42,
    verbose=False
)

print('⏳ Entrenando Red Neuronal...')
nn_model.fit(X_train, y_train_encoded)
print('✓ Entrenamiento completado')

# Predecir
y_pred_nn = nn_model.predict(X_test)

# Métricas
f1_nn = f1_score(y_test_encoded, y_pred_nn, average='macro')
f1_weighted_nn = f1_score(y_test_encoded, y_pred_nn, average='weighted')

print(f'\n📊 RESULTADOS:')
print(f'  F1-Score Macro:    {f1_nn:.4f}')
print(f'  F1-Score Weighted: {f1_weighted_nn:.4f}')

## 5. COMPARACIÓN DE MODELOS

In [ ]:
# Crear DataFrame con resultados
resultados = pd.DataFrame({
    'Modelo': ['Random Forest', 'Gradient Boosting', 'XGBoost', 'LightGBM', 'SVM', 'Neural Network'],
    'F1-Macro': [f1_rf, f1_gb, f1_xgb, f1_lgb, f1_svm, f1_nn],
    'F1-Weighted': [f1_weighted_rf, f1_weighted_gb, f1_weighted_xgb, 
                    f1_weighted_lgb, f1_weighted_svm, f1_weighted_nn]
}).sort_values('F1-Macro', ascending=False)

print('\n' + '='*80)
print('COMPARACIÓN DE TODOS LOS MODELOS')
print('='*80)
print(resultados.to_string(index=False))

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: F1-Macro
axes[0].barh(resultados['Modelo'], resultados['F1-Macro'], color='steelblue')
axes[0].set_xlabel('F1-Score Macro', fontweight='bold')
axes[0].set_title('Comparación por F1-Macro', fontweight='bold', fontsize=14)
axes[0].grid(axis='x', alpha=0.3)

# Gráfico 2: F1-Weighted
axes[1].barh(resultados['Modelo'], resultados['F1-Weighted'], color='coral')
axes[1].set_xlabel('F1-Score Weighted', fontweight='bold')
axes[1].set_title('Comparación por F1-Weighted', fontweight='bold', fontsize=14)
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. SELECCIÓN DEL MEJOR MODELO

In [ ]:
# Diccionario con todos los modelos
modelos_dict = {
    'Random Forest': (rf, f1_rf),
    'Gradient Boosting': (gb, f1_gb),
    'XGBoost': (xgb_model, f1_xgb),
    'LightGBM': (lgb_model, f1_lgb),
    'SVM': (svm_model, f1_svm),
    'Neural Network': (nn_model, f1_nn)
}

# Seleccionar el mejor
best_name = max(modelos_dict, key=lambda x: modelos_dict[x][1])
best_model = modelos_dict[best_name][0]
best_f1 = modelos_dict[best_name][1]

print('\n' + '='*80)
print('🏆 MEJOR MODELO SELECCIONADO')
print('='*80)
print(f'\n  Modelo: {best_name}')
print(f'  F1-Score Macro: {best_f1:.4f}')
print('\n' + '='*80)

## 7. EVALUACIÓN DETALLADA DEL MEJOR MODELO

In [ ]:
# Predicciones del mejor modelo
y_pred_best = best_model.predict(X_test)

# Decodificar para mostrar nombres de clases
y_test_decoded = label_encoder.inverse_transform(y_test_encoded)
y_pred_decoded = label_encoder.inverse_transform(y_pred_best)

# Reporte de clasificación
print('\n' + '='*80)
print(f'REPORTE DE CLASIFICACIÓN - {best_name}')
print('='*80)
print(classification_report(y_test_decoded, y_pred_decoded))

## 8. MATRIZ DE CONFUSIÓN

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test_decoded, y_pred_decoded, labels=label_encoder.classes_)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_,
            cbar_kws={'label': 'Número de casos'})
plt.title(f'Matriz de Confusión - {best_name}', fontweight='bold', fontsize=14, pad=15)
plt.ylabel('Clase Real', fontweight='bold')
plt.xlabel('Clase Predicha', fontweight='bold')
plt.tight_layout()
plt.show()

# Análisis de la matriz
print('\n💡 INTERPRETACIÓN DE LA MATRIZ:')
print('  - Diagonal: Predicciones correctas')
print('  - Fuera de diagonal: Confusiones entre clases')
print('  - Observar qué clases se confunden más entre sí')

## 9. GUARDAR MODELO FINAL Y ENCODER

In [ ]:
# Guardar mejor modelo
with open('modelo_final.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Ya guardamos el encoder anteriormente, pero reconfirmamos
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print('\n' + '='*80)
print('✅ ARCHIVOS GUARDADOS')
print('='*80)
print('\n  1. modelo_final.pkl      → Mejor modelo entrenado')
print('  2. label_encoder.pkl     → Encoder para decodificar predicciones')

print('\n💡 USO EN PRODUCCIÓN:')
print('  # Cargar modelo')
print('  with open("modelo_final.pkl", "rb") as f:')
print('      modelo = pickle.load(f)')
print('  ')
print('  # Cargar encoder')
print('  with open("label_encoder.pkl", "rb") as f:')
print('      encoder = pickle.load(f)')
print('  ')
print('  # Predecir')
print('  pred_encoded = modelo.predict(X_new)')
print('  pred_decoded = encoder.inverse_transform(pred_encoded)')
print('  # pred_decoded contendrá los nombres: "CANCER", "VIH", etc.')

print('\n' + '='*80)

## 10. RESUMEN Y CONCLUSIONES

---

### ✅ Logros:
1. Entrenamiento exitoso de 6 modelos diferentes
2. Comparación objetiva con métricas balanceadas
3. Selección del mejor modelo basado en F1-Score macro
4. Evaluación detallada con matriz de confusión
5. Modelo guardado y listo para producción

### 📊 Resultados Esperados:
- **F1-Score Macro**: > 0.75 (balance entre todas las clases)
- **F1-Score Weighted**: > 0.80 (considerando distribución real)
- **Recall por clase**: Idealmente > 0.70 para cada enfermedad

### 🎯 Mejor Modelo:
- Típicamente XGBoost o LightGBM obtienen los mejores resultados
- Buen balance entre velocidad y precisión
- Interpretable con feature importance

### ⚠️ Consideraciones Importantes:
1. **LabelEncoder es CRÍTICO**: Sin él, XGBoost y LightGBM fallan
2. **Guardar el encoder**: Necesario para decodificar predicciones
3. **Test set sin balancear**: Refleja la distribución real
4. **Métricas balanceadas**: F1-Macro más importante que accuracy

### 🚀 Próximos Pasos:
1. **Notebook 07**: Interpretación con SHAP values
2. **Notebook 08**: Aplicación web con Streamlit

---